# Euclidean Attack on RSA

This program was made for an assignment of the class ”Cryptography” of my master’s program considering the study of the Euclidean Attack on RSA, as described on "An Attack on Small Private Keys of RSA Based on Euclidean Algorithm" of D. Poulakis, which can be found [here](https://eprint.iacr.org/2019/283).

Although more theoretical analysis has been done on another file, our main goal here is to create a program that can emulate the "EUCLID-ATTACK" algorithm for the factorization of n, proposed in the same paper.

<u>**EUCLID-ATTACK**</u>  
$\cdot$ Input: An RSA public key $(n,e)$ with $e > n \, / \, c$  
$\cdot$ Output: The primes $p$ and $q$ (or FAIL)
1. Compute $a=(n+1) \, mod \, e$
2. Using the extended Euclidean algorithm for $e$ and $a$, compute the biggest remainder $r_j$ among them which are $<e^{3/4}$,  
   as well as the associated integers $s_j, t_j$ such that $s_je+at_j=r_j$.
3. Compute $\mu_j = gcd(t_j,r_j)$ and next $t'_j=t_j \, / \, \mu_j$.
4. Compute $\beta_1 = (a+|t'_j|^{-1}) \, mod \, e$ and next the solutions $u_1$ and $v_1$ of equation $x^2-\beta_1x+n=0$.  
   If $u_1$ and $v_1$ are positive integers, then output $(u_1, v_1)$. Otherwise, go to the next step.
5. Compute $\beta_2 = (a+(e-|t'_j|)^{-1}) \, mod \, e$ and next the solutions $u_2$ and $v_2$ of equation $x^2-\beta_2x+n=0$.  
   If $u_2$ and $v_2$ are positive integers, then output $(u_2, v_2)$. Otherwise, output FAIL.

##### Notes:
1. One change that we will make to the algorithm in our implementation is that the output will be more than the end results. We would also like to compute the inbetween values used. This is why we will compute both $(u_1, v_1)$ and $(u_2, v2)$, even if $(u_1, v_1)$ is already the right answer.  
2. Due to some coding issues concerning the nature of the solutions $(u_1, v_1)$ and $(u_2, v2)$ of the above quadratic equations for large numbers, small changes to the algorithm had to be made. More precisely, the validation process to see if the solutions are indeed the primes $p$ and $q$ will now be a check to see if $int(u_1)\cdot int(v_1)=n$ or $int(u_2) \cdot int(v_2)=n$ respectively, where $int()$ is a python function used to convert a specified value (any number or string) into an integer object. Hence, $(int(u_1), int(v_1))$ or $(int(u_2),int(v_2))$ will be output respectively in each case.

Author: Florias Papadopoulos

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-modules" data-toc-modified-id="Importing-modules-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing modules</a></span></li><li><span><a href="#Defining-the-functions" data-toc-modified-id="Defining-the-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Defining the functions</a></span></li><li><span><a href="#Solving-the-problem" data-toc-modified-id="Solving-the-problem-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Solving the problem</a></span></li></ul></div>

## Importing modules

We start by importing the modules that we will use

In [1]:
import math
from sympy import solve, N
from sympy.abc import x

If one or more modules are missing you can just type the code below in order to install a pip package in the current Jupyter kernel. For example, if SymPy is missing, then we can use

In [2]:
import sys
!{sys.executable} -m pip install sympy


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: C:\Users\HP\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


## Defining the functions

#### (a) modifiedExtendedEuclid

The first function that we will need is one that returns the results that we need in the 2nd step of the algorithm. More specifically, by inputting $(e, e, a)$ to the function below, the extended Euclidean algorithm for $e$ and $a$ will be performed. Moreover, the biggest remainder $r_j$ among them which are $<e^{3/4}$ will be computed, along with the integers $s_j, t_j$. All those will be necessary for the next, main function.

In [3]:
def modifiedExtendedEuclid(e, x, y):
    #initializing
    remainders_list = [x,y]
    wantedRemainders_list = []

    s0, s1 = 1, 0
    t0, t1 = 0, 1
    s_list = [s0, s1]
    t_list = [t0, t1]
    
    #making sure that x>=y
    if x < y:
        return modifiedExtendedEuclid(y, x)

    #main_loop
    i = 0
    while y != 0:
        i = i + 1
        q_iplus1 = x//y
        s_iplus1 = s_list[i-1] - s_list[i]*q_iplus1
        t_iplus1 = t_list[i-1] - t_list[i]*q_iplus1
        #print(i)
        #print('%s = %s * %s + %s' % (x, q_iplus1, y, x % y))
        #print('%s = %s - %s * %s' % (s_iplus1, s_list[i-1], s_list[i], q_iplus1))
        #print('%s = %s - %s * %s' % (t_iplus1, t_list[i-1], t_list[i], q_iplus1))
        
        s_list.append(s_iplus1)
        t_list.append(t_iplus1)
        remainders_list.append(x % y)
        (x, y) = (y, x % y)

    gcd = x

    #remove element from run where 
    s_list.pop()
    t_list.pop()
    remainders_list.pop()

    for remainder in remainders_list:
        if remainder < e**(3/4):
            wantedRemainders_list.append(remainder)

    if bool(wantedRemainders_list) == False:
        return "Found no remainders under e^(3/4)."
    else:
        r_j = max(wantedRemainders_list)
        j = remainders_list.index(r_j) 
        s_j = s_list[j]
        t_j = t_list[j]
    
    return remainders_list, s_list, t_list, gcd, j, r_j, s_j, t_j

#### (a) modEuclidAttack

This is the main function that carries out the EUCLID-ATTACK algorithm, using the previous function in the 2nd step. As mentioned before, only the RSA modulus $n$ and the derived $e$ need to be inputted and the function will return all inbetween results, not only the primes $p$ and $q$.

##### Notes. 
1. Due to a problem in naming variables, $t'_j$ is denoted as "clean_t_j".
2. Although everything else is relatively clear, we note that the solution of a quadratic equation with large coefficients proved to be a bigger problem than expected. Classical self-made codes could not give an accurate enough result for the validation, so the functions "solve()" and "N()" from SymPy were used. More precisely, "N()" was used with the option 100 after solve, which means that the numerical evaluation of the result of solve is performed to an accuracy of 100 decimal digits, in order to make sure that all digits will be intact in the procedures. If there is a problem with the algorithm, one should first try to upper this option and then try other changes.

In [4]:
def modEuclidAttack(n,e):
    
    #step1
    a = (n+1) % e

    #step2
    remainders_list, s_list, t_list, gcd, j, r_j, s_j, t_j  = modifiedExtendedEuclid(e, e, a)

    #step3 
    mu_j = math.gcd(t_j,r_j)
    clean_t_j = t_j // mu_j

    #step4
    inv1 = pow(clean_t_j,-1,e)
    b1 = (a+inv1) % e

    f1 = x**2 - b1*x + n
    f1_roots = solve(f1, x)
    u1_exact = N(f1_roots[0],100)
    v1_exact = N(f1_roots[1],100)
    u1 = N(f1_roots[0])
    v1 = N(f1_roots[1])

    #step5
    inv2 = pow(e-abs(clean_t_j),-1,e)
    b2 = (a+inv2) % e

    f2 = x**2 - b2*x + n
    f2_roots = solve(f2, x)
    u2_exact = N(f2_roots[0],100)
    v2_exact = N(f2_roots[1],100)
    u2 = N(f2_roots[0])
    v2 = N(f2_roots[1])

    #validation
    if int(u1_exact)*int(v1_exact) == n:
        solution1 = int(u1_exact)
        solution2 = int(v1_exact)
    elif int(u2_exact)*int(v2_exact) == n:
        solution1 = int(u2_exact)
        solution2 = int(v2_exact)
    else:
        solution1 = "FAIL"
        solution2 = "FAIL"

    return a, remainders_list, s_list, t_list, gcd, j, r_j, s_j, t_j, mu_j, clean_t_j, inv1, b1, u1, u1_exact,\
           v1, v1_exact, inv2, b2, u2, u2_exact, v2_exact, v2, solution1, solution2

## Solving the problem

As an example of use of the main function, we now create a script that takes as input $n$ and $e$ from the public key 

$$(n,e)=(85080976323951696719635578579671062429, \, 61100559406251463256709716070302151015)$$ 

and outputs the process and the final result.  
For the validation, we note that the two primes which make up $n$ are $9223372036854777017 \text{ and } 9224497936761618437$.   
Moreover, we have  

$$\phi(n) = (9223372036854777017-1)(9224497936761618437-1) = 85080976323951696701187708606054666976$$

and the private key $d$, such that $d \equiv e^{-1} \, (mod \, \phi(n)$, is  

$$d=85080976323951696701187708606050456215$$

We note that $2e>n$ and so, $c=2$. Furthermore, we have $k=61100559406251463115431885554434703360$ which gives us $e-k=e-\frac{(ed-1)}{\phi(n)}>e^{1/4}/6\sqrt{2}$ and thus, we see that we have a result even if some conditions are not met. This might be a hint that the bounds mentioned in the theory of the algorithm can be extended further.

In [5]:
#input

n = 85080976323951696719635578579671062429
e = 61100559406251463256709716070302151015
#d= 85080976323951696701187708606050456215 (for size comparison)

#input

#main function 
a, remainders_list, s_list, t_list, gcd, j, r_j, s_j, t_j, mu_j, clean_t_j, inv1, b1, u1, u1_exact, v1,\
 v1_exact, inv2, b2, u2, u2_exact, v2_exact, v2, solution1, solution2 = modEuclidAttack(n,e)

#script
print("EUCLID-ATTACK algorithm is used to factorize n:")
print("")
print("In step 1, we compute a = (n+1) mod e = " + str(a) + ".")
print("In step 2, we use the extended Euclidean algorithm for $e$ and $a$, and calculate the biggest remainder r_j <e^{3/4}.")
print("           - r_" + str(j) + " = " + str(r_j) + " with j = " + str(j))
print("           Moreover, we compute its associated integers s_" + str(j) + " and t_" + str(j) + ":")
print("           - s_" + str(j) + " = " + str(s_j))
print("           - t_" + str(j) + " = " + str(t_j))
print("In step 3, we compute μ_" + str(j) + " = gcd(t_" + str(j) + ",r_" + str(j) + ") and next t'_" + str(j) + "= t_" +\
     str(j) + "/μ_" + str(j) + ", where:")
print("           - μ_" + str(j) + " = " + str(mu_j))
print("           - t'_" + str(j) + " = " + str(clean_t_j))
print("In step 4, first we compute β1 = (a+|t'_" + str(j) + "|^{-1}) mod e, and next find the roots u1 and v1 of x^2-xβ1+n.")
print("           - β1 = " + str(b1))
print("           - u1 = " + str(u1))
print("           - int(u1) =" + str(u1_exact))
print("           - v1 = " + str(v1))
print("           - int(v1) =" + str(v1_exact))
print("In step 5, first we compute β2 = (a+(e-|t'_" + str(j) + "|)^{-1}) mod e, and next find the roots u2 and v2 of x^2-xβ2+n.")
print("           - β2 = " + str(b2))
print("           - u2 = " + str(u2))
print("           - int(u2) =" + str(u2_exact))
print("           - v2 = " + str(v2))
print("           - int(v2) =" + str(v2_exact))
print("Finally, we check if int(u1)*int(v1)=n or int(u2)*int(v2)=n (see note 2 from first cell).")
print("If none of them are true, then the final output will be fail.")
print("")
if isinstance(solution1, str) and isinstance(solution2, str):
    print(">> FAIL")
else:
    print(">> The primes are " + str(solution1) + " and " + str(solution2) + ".")

EUCLID-ATTACK algorithm is used to factorize n:

In step 1, we compute a = (n+1) mod e = 23980416917700233462925862509368911415.
In step 2, we use the extended Euclidean algorithm for $e$ and $a$, and calculate the biggest remainder r_j <e^{3/4}.
           - r_13 = 55785270375887536485564215 with j = 13
           Moreover, we compute its associated integers s_13 and t_13:
           - s_13 = -1186820
           - t_13 = 3023941
In step 3, we compute μ_13 = gcd(t_13,r_13) and next t'_13= t_13/μ_13, where:
           - μ_13 = 1
           - t'_13 = 3023941
In step 4, first we compute β1 = (a+|t'_13|^{-1}) mod e, and next find the roots u1 and v1 of x^2-xβ1+n.
           - β1 = 47960833835400466907403855045121427376
           - u1 = 1.77396782999949
           - int(u1) =1.773967829999494447048988704657479652626631034643991945471243506155632617907731005515961807231068396
           - v1 = 4.79608338354005e+37
           - int(v1) =47960833835400466907403855045121427374.2260321700005055